# Environment Setting Up

In [42]:
import os
from dotenv import load_dotenv

# Loading environment variables from .env
load_dotenv()

# Changing directory to main directory for easy data access
working_directory = os.getenv("WORKING_DIRECTORY")
os.chdir(working_directory)

# Checking the change
%pwd

'D:\\Projects\\Stock Screener\\Stock-Screener-Agent'

In [43]:
from pathlib import Path

# Checking the change
print("Git folder exists:", Path(".git").exists())

Git folder exists: True


# 2. Fetch Data

## Stock Price Data

In [44]:
# Cache Manager
# Avoid redundant API calls by checking last fetch timestamps

from datetime import datetime, timedelta
from StockScreener.utils.common import load_json, save_json, create_directories
import json

# Define where cache index JSON will be stored.
# Uses the working_directory environment variable if available.
WORK_DIR = Path(working_directory) / "artifacts"
CACHE_PATH = WORK_DIR / "cache_index.json"

# Define how long cache entries remain valid.
CACHE_EXPIRY_HOURS = 6

def update_cache_index(ticker: str, data_type: str) -> None:
    """
    Update or create a cache entry for the given ticker and data type.

    Args:
        ticker (str): Stock symbol (e.g., "AAPL").
        data_type (str): Type of data cached (e.g., "price", "news").
    """
    # Ensure the cache directory exists
    create_directories([WORK_DIR])

    ticker = ticker.upper()

    # Load the cache file if it exists, else create an empty dict
    try:
        cache = load_json(CACHE_PATH)
    except:
        cache = {}

    # Update timestamp for this ticker + data_type
    cache.setdefault(ticker, {})[data_type] = datetime.now().isoformat()

    # Save back to JSON
    save_json(save_path=CACHE_PATH, data=cache)


def is_cache_stale(ticker: str, data_type: str) -> bool:
    """
    Determine if cache for a given ticker and data_type is expired or missing.

    Args:
        ticker (str): Stock symbol (e.g., "AAPL").
        data_type (str): Type of data cached (e.g., "price", "news").

    Returns:
        bool: True if cache is stale or missing, False if still valid.
    """
    # Ensure the cache directory exists
    create_directories([WORK_DIR])

    ticker = ticker.upper()
    
    # Load cache safely
    try:
        cache = load_json(CACHE_PATH)
        entry = cache.get(ticker, {}).get(data_type)
    except (FileNotFoundError, json.JSONDecodeError):
        return True

    # If no entry found, it's stale
    if not entry:
        return True

    # Parse last updated timestamp and compare with expiry threshold
    last_updated = datetime.fromisoformat(entry)
    return datetime.now() - last_updated > timedelta(hours=CACHE_EXPIRY_HOURS)

In [45]:
import yfinance as yf
import pandas as pd

def fetch_prices(ticker: str):
    ticker = ticker.upper()
    load_path = f"artifacts/prices/{ticker}.pkl"

    if Path(load_path).exists() and not is_cache_stale(ticker, "prices"):
        return pd.read_pickle(load_path)
    
    df = yf.download(ticker, period="1y", interval="1d", auto_adjust=True)
    df.to_pickle(load_path)
    update_cache_index(ticker, "prices")
    return df

In [46]:
fetch_prices(ticker="AAPL")

[*********************100%***********************]  1 of 1 completed

[2025-10-27 18:17:49,932: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:17:49,932: ERROR: common: JSON file not found at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:17:49,932: ERROR: common: File not found: JSON file not found at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:17:49,932: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:17:49,932: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2024-10-25,230.338013,232.139626,228.506540,228.675751,38802300
2024-10-28,232.318787,233.642627,231.472733,232.239171,36087100
2024-10-29,232.587540,233.244486,231.243803,232.020188,35417200
2024-10-30,229.034088,232.388472,228.486633,231.532455,47070900
2024-10-31,224.863480,228.765318,224.325973,228.277583,64370100
...,...,...,...,...,...
2025-10-20,262.239990,264.380005,255.630005,255.889999,90483000
2025-10-21,262.769989,265.290009,261.829987,261.880005,46695900


## Stock Fundamental Data

In [47]:
stock = yf.Ticker("AAPL")
info = stock.info  # Pulls full fundamental data
info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [48]:
stock = yf.Ticker("AAPL")
info = stock.info
info.get("sector").lower()

'technology'

In [49]:
import yfinance as yf

def get_fundamental_data(ticker: str) -> dict:
    ticker = ticker.upper()
    stock = yf.Ticker(ticker)
    info = stock.info  # Pulls full fundamental data
    
    # Select only relevant keys
    fundamentals = {
        "PE": info.get("trailingPE"),
        "PB": info.get("priceToBook"),
        "PS": info.get("priceToSalesTrailing12Months"),
        "EV_EBITDA": info.get("enterpriseToEbitda"),
        "ROE": info.get("returnOnEquity"),
        "EPS": info.get("trailingEps"),
        "Book Value": info.get("bookValue"),
        "Sector": info.get("sector").lower(),
    }

    return fundamentals

df = get_fundamental_data(ticker="AAPL")
df

{'PE': 39.821213,
 'PB': 59.313923,
 'PS': 9.545063,
 'EV_EBITDA': 27.853,
 'ROE': 1.49814,
 'EPS': 6.6,
 'Book Value': 4.431,
 'Sector': 'technology'}

In [50]:
import yfinance as yf

def get_fundamental_data(ticker: str) -> dict:
    ticker = ticker.upper()
    stock = yf.Ticker(ticker)
    info = stock.info  # Pulls full fundamental data
    
    # Select only relevant keys
    fundamentals = {
        "PE": info.get("trailingPE", "Not Found"),
        "PB": info.get("priceToBook", "Not Found"),
        "PS": info.get("priceToSalesTrailing12Months", "Not Found"),
        "EV_EBITDA": info.get("enterpriseToEbitda", "Not Found"),
        "ROE": info.get("returnOnEquity", "Not Found"),
        "EPS": info.get("trailingEps", "Not Found"),
        "Book Value": info.get("bookValue", "Not Found"),
        "Sector": info.get("sector", "Not Found").title(),
    }

    return fundamentals


def fetch_fundamentals(ticker: str):
    ticker = ticker.upper()
    load_path = WORK_DIR / f"fundamentals/{ticker}.json"

    if Path(load_path).exists() and not is_cache_stale(ticker, "fundamentals"):
        return load_json(load_path)
    
    data = get_fundamental_data(ticker)

    save_json(save_path=load_path, data=data)
    update_cache_index(ticker, "fundamentals")
    
    return data

In [51]:
fetch_fundamentals(ticker="AAPL")

[2025-10-27 18:17:50,734: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\fundamentals created successfully.]
[2025-10-27 18:17:50,734: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\fundamentals\AAPL.json]
[2025-10-27 18:17:50,734: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:17:50,749: INFO: common: JSON file succesfully loaded form: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:17:50,749: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:17:50,749: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]


{'PE': 39.821213,
 'PB': 59.313923,
 'PS': 9.545063,
 'EV_EBITDA': 27.853,
 'ROE': 1.49814,
 'EPS': 6.6,
 'Book Value': 4.431,
 'Sector': 'Technology'}

## Stock Sentiment Data

In [52]:
from transformers import pipeline
import requests
import os
from newspaper import Article
import trafilatura


def get_feed(ticker: str, look_back: int=7):
    ticker = ticker.upper()
    api_key = os.getenv("NEWS_API_KEY")
    url = f"https://newsapi.org/v2/everything?q={ticker}&from={(datetime.now()-timedelta(days=look_back)).date()}&apiKey={api_key}"
    res = requests.get(url).json()
    articles = res.get("articles", [])
    return articles


def fetch_article_text(url: str):
    try:
        article = Article(url)
        article.download()
        article.parse()
        if len(article.text.strip()) < 200:  # Too short, likely failed
            raise ValueError("Incomplete text")
        return article.text
    
    except Exception as e:
        print(f"Failed to parse {url}: {e}")
        return None


def get_content(article):
    url = article.get("url", None)

    if url:
        text = fetch_article_text(url)
        if text:
            return text
        try:
            downloaded = trafilatura.fetch_url(url)
            return trafilatura.extract(downloaded)
        except:
            pass
    
    return article.get("title" + ". ", "") + article.get("description", "")


def fetch_sentiment(ticker: str):
    ticker = ticker.upper()
    load_path = WORK_DIR / f"sentiment/{ticker}.json"

    if Path(load_path).exists() and not is_cache_stale(ticker, "sentiment"):
        return load_json(load_path)

    # Creating the sentiment analysis pipeline
    classifier = pipeline(task='text-classification', model='ProsusAI/finbert', device='cpu')
    
    ticker_feed = get_feed(ticker=ticker, look_back=10)

    total = 0
    num_articles = 0
    positive = 0
    total_articles = 0
    all_content = []

    for article in ticker_feed:
        title = article.get("title", "") or ""
        desc = article.get("description", "") or ""
        content = f"{title}. {desc}".strip()

        if content:
            sentiment = classifier(content)[0]
            total_articles += 1

            if sentiment["label"] == 'positive' or sentiment["label"] == 'negative':
                total += sentiment["score"]
                num_articles += 1
                all_content.append(content)

            if sentiment["label"] == "positive":
                positive += 1

    data = {"Overall Sentiment": total / num_articles,
            "Positive Articles": positive,
            "Negative Artciles": num_articles - positive,
            "Total Artciles": total_articles,
            "Category": "Positive" if total >= 0.2 else "Negative" if total <= 0.2 else "Neutral",
            "Contents": all_content,}

    save_json(save_path=load_path, data=data)
    update_cache_index(ticker, "sentiment")
    return data


In [53]:
arts = get_feed("aapl")

In [54]:
arts

[{'source': {'id': None, 'name': 'Yahoo Entertainment'},
  'author': 'Yahoo Finance Video',
  'title': '3 most important things to watch with Big Tech earnings next week',
  'description': 'Big Tech earnings are on deck next week with Magnificent Seven companies Microsoft (MSFT), Amazon (AMZN), Apple (AAPL), Alphabet (GOOG, GOOGL), and Meta...',
  'url': 'https://finance.yahoo.com/video/3-most-important-things-watch-205449651.html',
  'urlToImage': 'https://s.yimg.com/ny/api/res/1.2/yiQBY.8cGY1HQDxIiUwb3g--/YXBwaWQ9aGlnaGxhbmRlcjt3PTEyMDA7aD02NzU-/https://s.yimg.com/os/creatr-uploaded-images/2025-10/9ea40cb0-b11b-11f0-badf-defbcab889b0',
  'publishedAt': '2025-10-24T20:54:49Z',
  'content': 'Well Big tech taking center stage. Microsoft, Alphabet, Meta reporting earnings on Wednesday followed by Apple and Amazon on Thursday. Together, they represent a huge slice of the market, saying the … [+6287 chars]'},
 {'source': {'id': None, 'name': '9to5Mac'},
  'author': 'Ryan Christoffel',
  't

In [55]:
from newspaper import Article

url = arts[0]["url"]
article = Article(url)
article.download()
article.parse()
print(article.text)

00:00 Speaker A

Well Big tech taking center stage. Microsoft, Alphabet, Meta reporting earnings on Wednesday followed by Apple and Amazon on Thursday. Together, they represent a huge slice of the market, saying the tone for stocks into year end. We're breaking down the three most important things to watch heading into these earnings. And joining us now we got Gil Luria, head of Technology research at D.A. Davidson. Gil, always great to see you. So big tech earnings on deck, Gil.

00:30 Speaker A

Uh Big tech investors have plenty of questions about valuations, about the AI spending boom. Maybe uh maybe let let's start there, Gil. Let let's start on this question. AI boom, AI bubble. I'm sure you're getting questions from clients about that, Gil. What are you telling them?

00:54 Gil Luria

That as far as the companies that report next week go, there's no bubble. Microsoft, Amazon, Google and Meta are spending most of the capital expenditures on AI, but they're doing it to match demand

In [56]:
type(article.text)

str

In [57]:
import trafilatura

downloaded = trafilatura.fetch_url(url)
content = trafilatura.extract(downloaded)
content

'Big Tech earnings are on deck next week with Magnificent Seven companies Microsoft (MSFT), Amazon (AMZN), Apple (AAPL), Alphabet (GOOG, GOOGL), and Meta Platforms (META) set to report quarterly results.\nD.A. Davidson Head of Technology Research Gil Luria explains to Josh Lipton where he is seeing "bubblicious aspects" begin to form around the AI trade and Big Tech\'s capex investments into artificial intelligence, including Nvidia\'s (NVDA) strategic partnership with OpenAI (OPAI.PVT), while also commenting on Meta and Google\'s advertising businesses.\nTo watch more expert insights and analysis on the latest market action, check out more Market Domination.\nWell Big tech taking center stage. Microsoft, Alphabet, Meta reporting earnings on Wednesday followed by Apple and Amazon on Thursday. Together, they represent a huge slice of the market, saying the tone for stocks into year end. We\'re breaking down the three most important things to watch heading into these earnings. And joinin

In [58]:
type(content)

str

In [59]:
print(len(article.text), len(content))

6824 7109


In [60]:
fetch_sentiment("aapl")

Device set to use cpu


[2025-10-27 18:18:01,070: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\sentiment created successfully.]
[2025-10-27 18:18:01,070: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\sentiment\AAPL.json]
[2025-10-27 18:18:01,070: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:01,070: INFO: common: JSON file succesfully loaded form: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:18:01,070: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:01,070: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]


{'Overall Sentiment': 0.8271641266696593,
 'Positive Articles': 25,
 'Negative Artciles': 9,
 'Total Artciles': 100,
 'Category': 'Positive',
 'Contents': ['Apple Stockholders Hit $850 Billion Jackpot. Over the past ten years, Apple stock has delivered a massive $847 billion back to its investors in tangible cash in the form of dividends and buybacks.',
  'Apple Shines, Leads Parade Of Megacap Hyperscaler Earnings Reports. Several megacap hyperscalers headline the latest earnings calendar, including Meta Platforms (META), Leaderboard stock Alphabet (GOOGL), Amazon.com (AMZN), Microsoft (MSFT) and Apple (AAPL). Apple stock is holding near a fresh buy point, helped by early signs…',
  'Wells Fargo Raises Its Price Target on Apple (AAPL) to $290, Maintains “Overweight” Rating. Apple Inc. (NASDAQ:AAPL) is one of the 13 best Fortune 500 stocks to invest in now. On October 21, 2025, Wells Fargo raised its price target on Apple Inc...',
  'Evercore ISI Maintains Bullish Outlook for Apple Inc.

## Benchmark Data

In [61]:
import yfinance as yf

ticker = "SPY"
stock = yf.Ticker(ticker)
info = stock.info  # Pulls full fundamental data

# Select only relevant keys
benchmark = {
    "PE": info.get("trailingPE", "Not Found"),
    "PB": info.get("priceToBook", "Not Found"),
    "Book Value": info.get("bookValue", "Not Found"),
}

benchmark

{'PE': 28.596827, 'PB': 1.5778621, 'Book Value': 429.22}

In [62]:
import yfinance as yf
from StockScreener.utils.common import read_yaml, load_json

def load_benchmark(ticker: str):
    ticker = ticker.upper()
    sector_path = Path("config/sector_map.yaml")
    sector_map = read_yaml(path=sector_path)

    ticker_sector = load_json(Path(f"artifacts/fundamentals/{ticker}.json")).Sector

    # Loading Technical and Fundamental data for sector ETF
    try:
        fetch_benchmark_fundamental(sector_map.sectors[ticker_sector])
        fetch_benchmark_technical(sector_map.sectors[ticker_sector])
    except:
        pass

    # Loading Technical and Fundamental data for market ETF
    try:
        fetch_benchmark_fundamental(sector_map.market)
        fetch_benchmark_technical(sector_map.market)
    except:
        pass


def fetch_benchmark_technical(ticker: str) -> None:
    ticker = ticker.upper()
    load_path = f"artifacts/benchmarks/{ticker}.pkl"

    if Path(load_path).exists() and not is_cache_stale(ticker, "benchmark_prices"):
        return pd.read_pickle(load_path)
    
    df = yf.download(ticker, period="1y", interval="1d", auto_adjust=True)
    df.to_pickle(load_path)
    update_cache_index(ticker, "benchmark_prices")
    return df


def get_benchmark_data(ticker: str) -> dict:
    ticker = ticker.upper()
    etf = yf.Ticker(ticker)
    info = etf.info  # Pulls full fundamental data
    
    # Select only relevant keys
    benchmark = {
        "PE": info.get("trailingPE", "Not Found"),
        "PB": info.get("priceToBook", "Not Found"),
        "Book Value": info.get("bookValue", "Not Found"),
    }

    return benchmark


def fetch_benchmark_fundamental(ticker: str):
    ticker = ticker.upper()
    load_path = WORK_DIR / f"benchmarks/{ticker}.json"

    if Path(load_path).exists() and not is_cache_stale(ticker, "benchmark_fundamental"):
        return load_json(load_path)
    
    data = get_benchmark_data(ticker)

    save_json(save_path=load_path, data=data)
    update_cache_index(ticker, "benchmark_fundamental")
    
    return data

In [63]:
sector = load_json(Path(f"artifacts/fundamentals/{'AAPL'}.json")).Sector

[2025-10-27 18:18:01,289: INFO: common: JSON file succesfully loaded form: artifacts\fundamentals\AAPL.json]


In [64]:
sector_map = read_yaml(path=Path("config/sector_map.yaml"))
sector_map.sectors[sector]

[2025-10-27 18:18:01,296: INFO: common: YAML file: config\sector_map.yaml loaded successfully]


'XLK'

In [65]:
load_benchmark("AAPL")

[2025-10-27 18:18:01,305: INFO: common: YAML file: config\sector_map.yaml loaded successfully]
[2025-10-27 18:18:01,305: INFO: common: JSON file succesfully loaded form: artifacts\fundamentals\AAPL.json]
[2025-10-27 18:18:01,498: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\benchmarks created successfully.]
[2025-10-27 18:18:01,498: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\benchmarks\XLK.json]
[2025-10-27 18:18:01,498: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:01,500: INFO: common: JSON file succesfully loaded form: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:18:01,500: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:01,501: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screen

[*********************100%***********************]  1 of 1 completed

[2025-10-27 18:18:01,686: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:01,688: INFO: common: JSON file succesfully loaded form: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:18:01,689: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:01,689: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]


[2025-10-27 18:18:02,264: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\benchmarks created successfully.]
[2025-10-27 18:18:02,265: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\benchmarks\SPY.json]
[2025-10-27 18:18:02,267: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:02,267: INFO: common: JSON file succesfully loaded form: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:18:02,267: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:02,267: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]


[*********************100%***********************]  1 of 1 completed

[2025-10-27 18:18:02,481: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:02,481: INFO: common: JSON file succesfully loaded form: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:18:02,481: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:02,481: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]


## Combined

In [66]:
def update_all(ticker):
    fetch_prices(ticker)
    fetch_fundamentals(ticker)
    fetch_sentiment(ticker)
    load_benchmark(ticker)

In [67]:
stock = yf.Ticker("tsla")
info = stock.info
info.get("sector").lower()

'consumer cyclical'

In [68]:
ticker = "TSLA"
update_all(ticker=ticker)

[*********************100%***********************]  1 of 1 completed

[2025-10-27 18:18:05,910: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:05,910: INFO: common: JSON file succesfully loaded form: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:18:05,911: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:05,912: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]


[2025-10-27 18:18:06,807: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\fundamentals created successfully.]
[2025-10-27 18:18:06,808: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\fundamentals\TSLA.json]
[2025-10-27 18:18:06,809: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:06,809: INFO: common: JSON file succesfully loaded form: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:18:06,810: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:06,811: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]


Device set to use cpu


[2025-10-27 18:18:12,458: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\sentiment created successfully.]
[2025-10-27 18:18:12,458: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\sentiment\TSLA.json]
[2025-10-27 18:18:12,458: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:12,464: INFO: common: JSON file succesfully loaded form: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:18:12,464: INFO: common: Directory: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts created successfully.]
[2025-10-27 18:18:12,464: INFO: common: JSON file saved at: D:\Projects\Stock Screener\Stock-Screener-Agent\artifacts\cache_index.json]
[2025-10-27 18:18:12,470: INFO: common: YAML file: config\sector_map.yaml loaded successfully]
[2025-10-27 18:18:12,481: INFO: common: JSON file succesfully loaded form

## Cleaned

In [69]:
from dataclasses import dataclass
from pathlib import Path
from StockScreener.utils.logger import get_logger

# Initializing the logger
logger = get_logger()

@dataclass(frozen=True)
class IngestionConfig:
    """
    Immutable configuration class to hold all necessary paths 
    and parameters required for the ingestion stage.
    """
    root_dir: Path


In [70]:
# TEMP INGESTION PATHS

'''

CACHE_PATH = "artifacts/cache_index.json"
CACHE_EXPIRY_HOURS = 6
working_directory = os.getenv("WORKING_DIRECTORY")


"artifacts/prices/{ticker}.pkl"
load_path = WORK_DIR / f"fundamentals/{ticker}.json"


load_path = WORK_DIR / f"sentiment/{ticker}.json"
api_key = os.getenv("NEWS_API_KEY")
url = f"https://newsapi.org/v2/everything?q={ticker}&from={(datetime.now()-timedelta(days=look_back)).date()}&apiKey={api_key}"


sector_path = Path("config/sector_map.yaml")
ticker_sector = load_json(Path(f"artifacts/fundamentals/{ticker}.json")).Sector


'''

'\n\nCACHE_PATH = "artifacts/cache_index.json"\nCACHE_EXPIRY_HOURS = 6\nworking_directory = os.getenv("WORKING_DIRECTORY")\n\n\n"artifacts/prices/{ticker}.pkl"\nload_path = WORK_DIR / f"fundamentals/{ticker}.json"\n\n\nload_path = WORK_DIR / f"sentiment/{ticker}.json"\napi_key = os.getenv("NEWS_API_KEY")\nurl = f"https://newsapi.org/v2/everything?q={ticker}&from={(datetime.now()-timedelta(days=look_back)).date()}&apiKey={api_key}"\n\n\nsector_path = Path("config/sector_map.yaml")\nticker_sector = load_json(Path(f"artifacts/fundamentals/{ticker}.json")).Sector\n\n\n'